# Application: Heterogeneous Effect of Sex on Wage Using Double Lasso

We use US census data from the year 2015 to analyse the effect of gender and interaction effects of other variables with gender on wage jointly. The dependent variable is the logarithm of the wage, the target variable is *female* (in combination with other variables). All other variables denote some other socio-economic characteristics, e.g. marital status, education, and experience.  



This analysis allows a closer look how the gender wage gap is related to other socio-economic variables.


In [ ]:
# Import relevant packages
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import patsy
import warnings
warnings.simplefilter('ignore')
np.random.seed(1234)

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
data = pd.read_csv(file)

In [ ]:
data.describe()

Define outcome and regressors

In [8]:
y = np.log(data['wage']).values
Z = data.drop(['wage', 'lwage'], axis=1)
Z.columns

Index(['sex', 'shs', 'hsg', 'scl', 'clg', 'ad', 'mw', 'so', 'we', 'ne', 'exp1',
       'exp2', 'exp3', 'exp4', 'occ', 'occ2', 'ind', 'ind2'],
      dtype='object')

## Feature Engineering

Construct all our control variables

In [9]:
# Ultra flexible controls of all pair-wise interactions (around 1k variables); un-comment to run this
Zcontrols = patsy.dmatrix('0 + (shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we+exp1+exp2+exp3+exp4)**2',
                          Z, return_type='dataframe')

Zcontrols = Zcontrols - Zcontrols.mean(axis=0)
Zcontrols.describe()

,C(occ2)[1],C(occ2)[2],C(occ2)[3],C(occ2)[4],C(occ2)[5],C(occ2)[6],C(occ2)[7],C(occ2)[8],C(occ2)[9],C(occ2)[10],...,we:exp1,we:exp2,we:exp3,we:exp4,exp1:exp2,exp1:exp3,exp1:exp4,exp2:exp3,exp2:exp4,exp3:exp4
count,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,...,5150.000000,5.150000e+03,5.150000e+03,5150.000000,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03,5.150000e+03
mean,8.968015e-18,-1.388318e-17,7.933244e-18,-2.759389e-18,1.379695e-18,4.828931e-18,-4.139084e-18,1.431433e-17,-2.759389e-18,1.138248e-17,...,0.000000,-4.415023e-17,-3.532018e-15,0.000000,-3.046366e-13,-6.735559e-13,-5.036658e-12,6.341739e-13,-9.819011e-14,-4.385354e-12
std,3.215556e-01,2.452604e-01,2.039887e-01,1.419958e-01,1.182224e-01,1.508723e-01,1.324702e-01,2.232981e-01,1.637613e-01,2.346088e-01,...,7.451711,2.200512e+00,7.370775e+00,26.078477,1.448896e+02,5.353022e+02,2.025738e+03,2.025738e+02,7.834032e+02,3.088170e+03
min,-1.170874e-01,-6.427184e-02,-4.349515e-02,-2.058252e-02,-1.417476e-02,-2.330097e-02,-1.786408e-02,-5.262136e-02,-2.757282e-02,-5.844660e-02,...,-2.979903,-6.439704e-01,-1.736658e+00,-5.256010,-8.235867e+01,-2.511804e+02,-8.174739e+02,-8.174739e+01,-2.777188e+02,-9.734162e+02
25%,-1.170874e-01,-6.427184e-02,-4.349515e-02,-2.058252e-02,-1.417476e-02,-2.330097e-02,-1.786408e-02,-5.262136e-02,-2.757282e-02,-5.844660e-02,...,-2.979903,-6.439704e-01,-1.736658e+00,-5.256010,-8.110867e+01,-2.505554e+02,-8.171614e+02,-8.171614e+01,-2.777031e+02,-9.734084e+02
50%,-1.170874e-01,-6.427184e-02,-4.349515e-02,-2.058252e-02,-1.417476e-02,-2.330097e-02,-1.786408e-02,-5.262136e-02,-2.757282e-02,-5.844660e-02,...,-2.979903,-6.439704e-01,-1.736658e+00,-5.256010,-7.235867e+01,-2.411804e+02,-8.074739e+02,-8.074739e+01,-2.767188e+02,-9.724162e+02
75%,-1.170874e-01,-6.427184e-02,-4.349515e-02,-2.058252e-02,-1.417476e-02,-2.330097e-02,-1.786408e-02,-5.262136e-02,-2.757282e-02,-5.844660e-02,...,-2.979903,-6.439704e-01,-1.736658e+00,-5.256010,1.025133e+01,-5.669938e+01,-4.090638e+02,-4.090638e+01,-1.919526e+02,-7.933074e+02
max,8.829126e-01,9.357282e-01,9.565049e-01,9.794175e-01,9.858252e-01,9.766990e-01,9.821359e-01,9.473786e-01,9.724272e-01,9.415534e-01,...,39.020097,1.699603e+01,7.235134e+01,305.913590,9.558713e+02,4.628501e+03,2.211703e+04,2.211703e+03,1.050150e+04,4.968890e+04


Construct all the variables that we will use to model heterogeneity of effect in a linear manner

In [ ]:
Zhet = patsy.dmatrix('0 + (shs+hsg+scl+clg+mw+so+we+exp1+exp2+exp3+exp4)',
                     Z, return_type='dataframe')
Zhet = Zhet - Zhet.mean(axis=0)

Construct all interaction variables between sex and heterogeneity variables

In [7]:
Zhet['sex'] = Z['sex']
Zinteractions = patsy.dmatrix('0 + sex + sex * (shs+hsg+scl+clg+mw+so+we+exp1+exp2+exp3+exp4)',
                              Zhet, return_type='dataframe')
interaction_cols = [c for c in Zinteractions.columns if c.startswith('sex')]

Put all the variables together

In [ ]:
X = pd.concat([Zinteractions, Zcontrols], axis=1)
X.shape

## Double Lasso for All Interactive Effects

We use "plug-in" tuning with a theoretically valid choice of penalty $\lambda = 2 \cdot c \hat{\sigma}\sqrt{n}  \Phi^{-1}(1-\alpha/2p)$, where $c>1$ and $1-\alpha$ is a confidence level, and $\Phi^{-1}$ denotes the quantile function. This choice ensures that the Lasso predictor is well behaved under independence as long as appropriate penalty weights are used.

In practice, many people choose to use cross-validation, which is perfectly fine for predictive tasks. However, when conducting inference, to make our analysis valid we will require cross-fitting in addition to cross-validation. As we have not yet discussed cross-fitting, we rely on this theoretically-driven penalty in order to allow for accurate inference in the upcoming notebooks.

Note: In the book, we multiply by $\sqrt{n}$. This is because there, Lasso minimizes the sum of errors. If you were using say sklearn's Lasso whose objective minimizes the average errors, you would instead divide by $\sqrt{n}$.

To estimate lasso using the theoretically motivated penalty level, we just use the hdmpy package. To install it run
```
!pip install multiprocess
!git clone https://github.com/maxhuppertz/hdmpy.git
```

You can run the cells below and then repeat the whole analysis above using the newly defined `lasso_model` variable.

In [ ]:
!pip install multiprocess
!git clone https://github.com/maxhuppertz/hdmpy.git

In [ ]:
import sys
sys.path.insert(1, "./hdmpy")

In [ ]:
# We wrap the package so that it has the familiar sklearn API
import hdmpy
from sklearn.base import BaseEstimator


class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])


def lasso_model():
    return RLasso(post=False)

In [ ]:
alpha = {}
res_y, res_D, epsilon = {}, {}, {}
for c in interaction_cols:
    print(f"Double Lasso for target variable {c}")
    D = X[c].values
    W = X.drop([c], axis=1)
    res_y[c] = y - lasso_model().fit(W, y).predict(W)
    res_D[c] = D - lasso_model().fit(W, D).predict(W)
    final = LinearRegression(fit_intercept=False).fit(res_D[c].reshape(-1, 1), res_y[c])
    epsilon[c] = res_y[c] - final.predict(res_D[c].reshape(-1, 1))
    alpha[c] = [final.coef_[0]]

# Calculate the covariance matrix of the estimated parameters
V = np.zeros((len(interaction_cols), len(interaction_cols)))
for it, c in enumerate(interaction_cols):
    Jc = np.mean(res_D[c]**2)
    for itp, cp in enumerate(interaction_cols):
        Jcp = np.mean(res_D[cp]**2)
        Sigma = np.mean(res_D[c] * epsilon[c] * epsilon[cp] * res_D[cp])
        V[it, itp] = Sigma / (Jc * Jcp)

# Calculate standard errors for each parameter
n = X.shape[0]
for it, c in enumerate(interaction_cols):
    alpha[c] += [np.sqrt(V[it, it] / n)]

# put all in a dataframe
df = pd.DataFrame.from_dict(alpha, orient='index', columns=['point', 'stderr'])

# Calculate and pointwise p-value
summary = pd.DataFrame()
summary['Estimate'] = df['point']
summary['Std. Error'] = df['stderr']
summary['p-value'] = norm.sf(np.abs(df['point'] / df['stderr']), loc=0, scale=1) * 2
summary['ci_lower'] = df['point'] - 1.96 * df['stderr']
summary['ci_upper'] = df['point'] + 1.96 * df['stderr']
summary

### Joint Confidence Intervals

In [ ]:
Drootinv = np.diagflat(1 / np.sqrt(np.diag(V)))
scaledCov = Drootinv @ V @ Drootinv
np.random.seed(123)
U = np.random.multivariate_normal(np.zeros(scaledCov.shape[0]), scaledCov, size=10000)
z = np.max(np.abs(U), axis=1)
c = np.percentile(z, 95)

summary = pd.DataFrame()
summary['Estimate'] = df['point']
summary['CI lower'] = df['point'] - c * df['stderr']
summary['CI upper'] = df['point'] + c * df['stderr']
summary